# 公式Example

深層学習フレームワークには公式に様々なモデルのExampleコードが公開されています。

## 【問題1】公式チュートリアルモデルを分担して実行
TensorFLowの公式チュートリアルモデルを分担して実行してください。


以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。


models/tutorials at master · tensorflow/models

###  [カスタムレイヤー](https://www.tensorflow.org/tutorials/customization/custom_layers#%E3%83%A2%E3%83%87%E3%83%AB%EF%BC%9A%E3%83%AC%E3%82%A4%E3%83%A4%E3%83%BC%E3%81%AE%E7%B5%84%E3%81%BF%E5%90%88%E3%82%8F%E3%81%9B)

ニューラルネットワークの構築には、ハイレベルの API である tf.keras を使うことを推奨します。しかしながら、TensorFlow API のほとんどは、eager execution でも使用可能です。

In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


### レイヤー：有用な演算の共通セット
機械学習モデルのコーディングでは、個々の演算やひとつひとつの変数のオペレーションよりは、より高度に抽象化されたオペレーションを行いたいのがほとんどだと思います。

多くの機械学習モデルは、比較的単純なレイヤーの組み合わせや積み重ねによって表現可能です。TensorFlow では、多くの一般的なレイヤーのセットに加えて、アプリケーションに特有なレイヤーを最初から記述したり、既存のレイヤーの組み合わせによって作るための、簡単な方法が提供されています。

TensorFlow には、tf.keras パッケージにKeras APIのすべてが含まれています。Keras のレイヤーは、独自のモデルを構築する際に大変便利です。

In [2]:
# tf.keras.layers パッケージの中では、レイヤーはオブジェクトです。
# レイヤーを構築するためにすることは、単にオブジェクトを作成するだけです。
# ほとんどのレイヤーでは、最初の引数が出力の次元あるいはチャネル数を表します。
#Denseは全結合層
layer = tf.keras.layers.Dense(100)

# 入力の次元数は多くの場合不要となっています。それは、レイヤーが最初に使われる際に
# 推定可能だからです。ただし、引数として渡すことで手動で指定することも可能です。
# これは複雑なモデルを構築する場合に役に立つでしょう。
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

既存のレイヤーのすべての一覧は、ドキュメントを参照してください。Dense（全結合レイヤー）、Conv2D、LSTM、BatchNormalization、Dropoutなどのたくさんのレイヤーが含まれています。

In [3]:
# レイヤーを使うには、単純にcallします。
layer(tf.zeros([10, 5]))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'dense_1/BiasAdd:0' shape=(10, 10) dtype=float32>

In [4]:
# レイヤーにはたくさんの便利なメソッドがあります。例えば、`layer.variables`を使って
# レイヤーのすべての変数を調べることができます。訓練可能な変数は、 `layer.trainable_variables`
# でわかります。この例では、全結合レイヤーには重みとバイアスの変数があります。
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32>]

In [5]:
# これらの変数には便利なアクセサを使ってアクセス可能です。
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32>)

### カスタムレイヤーの実装
独自のレイヤーを実装する最良の方法は、tf.keras.Layer クラスを拡張し、下記のメソッドを実装することです。

- __init__ , 入力に依存しないすべての初期化を行う
- build, 入力の shape を知った上で、残りの初期化を行う
- call, フォワード計算を行う

build が呼ばれるまで変数の生成を待つ必要はなく、__init__ で作成できることに注意してください。しかしながら、build で変数を生成することの優位な点は、レイヤーがオペレーションをしようとする入力の shape に基づいて、後から定義できる点です。これに対して、__init__ で変数を生成するということは、そのために必要な shape を明示的に指定する必要があるということです。

In [6]:
#tf.keras.layers.Layerを継承, サブクラス化してモデルを定義
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        #Python2の書き方
        #親クラスの初期化を行う
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    
    #重みを定義
    #self.kernelをadd_variableで上書き
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel", 
                                        shape=[int(input_shape[-1]), 
                                               self.num_outputs])
    #フォワード計算を行う
    #tf.matmulは内積
    def call(self, input):
        return tf.matmul(input, self.kernel)

#引数は出力の次元あるいはチャネル数
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
#trainable_variablesで訓練可能な変数を見ている
print(layer.trainable_variables)

Instructions for updating:
Please use `layer.add_weight` method instead.
Tensor("my_dense_layer/MatMul:0", shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32>]


できるだけ標準のレイヤーを使ったほうが、概してコードは読みやすく保守しやすくなります。コードを読む人は標準的なレイヤーの振る舞いに慣れているからです。tf.keras.layers にはないレイヤーを使いたい場合には、githubのイシューを登録するか、もっとよいのはプルリクエストを送ることです。

### モデル：レイヤーの組み合わせ
機械学習では、多くのレイヤーに類するものが、既存のレイヤーを組み合わせることで実装されています。例えば、[ResNet](https://deepage.net/deep_learning/2016/11/30/resnet.html)の残差ブロックは、畳込み、バッチ正規化とショートカットの組み合わせです。

他のレイヤーからなるレイヤーに類するものを定義する際の主役は、tf.keras.Model クラスです。このクラスを継承することで実装できます。



In [7]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://deepage.net/img/resnet/residual_block.jpg")

In [8]:
#tf.keras.Modelを継承,サブクラス化してモデルを定義
class ResnetIdentityBlock(tf.keras.Model):
    #親クラスの初期化を行う
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        #Conv2Dは2次元の畳み込みレイヤー
        #filters1は整数で出力空間の次元(畳み込みの出力フィルタの数)
        #(1, 1)は2次元の畳み込みウィンドウの幅と高さ
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        #BatchNormalizationは各バッチ毎に前の出力を正規化する(平均0標準偏差1)
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        #引数のkernel_sizeを第2引数にとる
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    
    #フォワードパスを定義
    def call(self, input_tensor, training=False):
        #入力は(batch_size, steps, input_dim)の3階テンソル
        #出力は(batch_size, new_steps, nb_filterの3階テンソル
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        #活性化関数はReLU関数
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        #残差ブロックは畳み込み層とshortcut connectionの組み合わせ
        x += input_tensor
        return tf.nn.relu(x)

    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

Tensor("resnet_identity_block/Relu_2:0", shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0']


しかし、ほとんどの場合には、モデルはレイヤーを次々に呼び出すことで構成されます。tf.keras.Sequential クラスを使うことで、これをかなり短いコードで実装できます。

In [9]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), 
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor 'sequential/batch_normalization_5/cond/Merge:0' shape=(1, 2, 3, 3) dtype=float32>

## 【問題2】（アドバンス課題）様々な手法を実行
TensorFLowやGoogle AI ResearchのGitHubリポジトリには、定番のモデルから最新のモデルまで多様なコードが公開されています。これらから興味あるものを選び実行してください。


なお、これらのコードは初学者向けではないため、巨大なデータセットのダウンロードが必要な場合など、実行が簡単ではないこともあります。そういった場合は、コードリーディングを行ってください。


models/research at master · tensorflow/models


google-research/google-research: Google AI Research


更新日が古いものはPythonやTensorFlowのバージョンが古く、扱いずらい場合があります。新しいものから見ることを推奨します。

# 異なるフレームワークへの書き換え

「ディープラーニングフレームワーク1」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。


- Iris（Iris-versicolorとIris-virginicaのみの2値分類）
- Iris（3種類全ての目的変数を使用して多値分類）
- House Prices
- MNIST

## Kerasへの書き換え
KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。


KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。


## 【問題3】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。



In [10]:
from keras import backend as K

In [11]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

Keras Sequential model API
1. Sequential()のインスタンスを作成し、レイヤをスタックする
2. Sequential()のインスタンスをcompileする
3. インスタンスにfitし、評価、predictする
4. インスタンスをsave

以下、Keras Sequential model APIの記法で、Sequential()のインスタンスを作成し、addメソッドでレイヤのインスタンスをスタックする書き方

In [12]:
#モデルを初期化
model = keras.models.Sequential()

#1つ目の隠れ層を追加
#10個の隠れユニットで構成
#入力層のためimput_dimの属性の値はデータセットの特徴量列の個数
model.add(keras.layers.Dense(10, input_dim=X_train.shape[1]))
#活性化関数はReLU
model.add(keras.layers.Activation('relu'))

#2つ目の隠れ層を追加
#連続する層のunitsとinput_dimの個数を一致させる
model.add(keras.layers.Dense(y_train.shape[1], input_dim=10))
#活性化関数はsigmoid
model.add(keras.layers.Activation('sigmoid')) 

#オプティマイザとコスト関数を指定してモデルをコンパイル
#binary_crossentropyはlog loss(2値分類の評価指標, クロスエントロピー)
#正解ラベルと予測値との距離を考慮した評価値
model.compile(optimizer='SGD',loss='binary_crossentropy')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
#fitメソッドを呼び出してモデルのトレーニングを行う
model.fit(X_train, y_train, nb_epoch=20, batch_size=5, verbose=1)

C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  



Epoch 1/20
64/64 [==============================] - 0s 4ms/step - loss: 0.9717
Epoch 2/20
64/64 [==============================] - 0s 254us/step - loss: 0.7825
Epoch 3/20
64/64 [==============================] - 0s 280us/step - loss: 0.7749
Epoch 4/20
64/64 [==============================] - 0s 180us/step - loss: 0.7701
Epoch 5/20
64/64 [==============================] - 0s 325us/step - loss: 0.7573
Epoch 6/20
64/64 [==============================] - 0s 369us/step - loss: 0.7499
Epoch 7/20
64/64 [==============================] - 0s 249us/step - loss: 0.7421
Epoch 8/20
64/64 [==============================] - 0s 296us/step - loss: 0.7470
Epoch 9/20
64/64 [==============================] - 0s 327us/step - loss: 0.7417
Epoch 10/20
64/64 [==============================] - 0s 280us/step - loss: 0.7404
Epoch 11/20
64/64 [==============================] - 0s 405us/step - loss: 0.7415
Epoch 12/20
64/64 [==============================] - 0s 375us/step - loss: 0.7384
Epoch 13/20
64/64 [=======

モデル評価

In [14]:
y_train_pred = model.predict_classes(X_train)
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]
print("Training Accuracy = {:.2f}".format(np.mean(train_acc)))

Training Accuracy = 0.53


In [15]:
y_val_pred = model.predict_classes(X_val)
correct_preds = np.sum(y_val == y_val_pred, axis=0)
val_acc = correct_preds / y_val.shape[0]
print("Val Accuracy = {:.2f}".format(np.mean(val_acc)))

Val Accuracy = 0.38


In [16]:
y_test_pred = model.predict_classes(X_test)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]
print("Test Accuracy = {:.2f}".format(np.mean(test_acc)))

Test Accuracy = 0.50


## 【問題4】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

# ワンホットライブラリのインスタンス作成
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [19]:
#レイヤの通し番号をリセット
K.clear_session()

#モデルを初期化
model = keras.models.Sequential()

#1つ目の隠れ層を追加
#12個の隠れユニットで構成
#入力層のためimput_dimの属性の値はデータセットの特徴量列の個数
model.add(keras.layers.Dense(10, input_dim=X_train.shape[1]))
#活性化関数はReLU
model.add(keras.layers.Activation('relu'))

#連続する層のunitsとinput_dimの個数を一致させる
model.add(keras.layers.Dense(y_train.shape[1], input_dim=10))
#活性化関数はSoftmax
model.add(keras.layers.Activation('softmax')) 

#オプティマイザとコスト関数を指定してモデルをコンパイル
model.compile(optimizer='SGD',loss='categorical_crossentropy')

In [20]:
#fitメソッドを呼び出してモデルのトレーニングを行う
model.fit(X_train, y_train, nb_epoch=20, batch_size=5, verbose=1)

C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
96/96 [==============================] - 0s 1ms/step - loss: 0.9484
Epoch 2/20
96/96 [==============================] - 0s 424us/step - loss: 0.7245
Epoch 3/20
96/96 [==============================] - 0s 283us/step - loss: 0.6696
Epoch 4/20
96/96 [==============================] - 0s 236us/step - loss: 0.6179
Epoch 5/20
96/96 [==============================] - 0s 284us/step - loss: 0.5831
Epoch 6/20
96/96 [==============================] - 0s 282us/step - loss: 0.5492
Epoch 7/20
96/96 [==============================] - 0s 308us/step - loss: 0.5266
Epoch 8/20
96/96 [==============================] - 0s 308us/step - loss: 0.5028
Epoch 9/20
96/96 [==============================] - 0s 329us/step - loss: 0.4896
Epoch 10/20
96/96 [==============================] - 0s 371us/step - loss: 0.4735
Epoch 11/20
96/96 [==============================] - 0s 333us/step - loss: 0.4552
Epoch 12/20
96/96 [==============================] - 0s 308us/step - loss: 0.4382
Epoch 13/20
96/96 [========

モデル評価

In [21]:
y_train_pred = model.predict_classes(X_train)
y_train_pred = enc.fit_transform(y_train_pred[:, np.newaxis])
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]
print("Training Accuracy = {:.2f}".format(np.mean(train_acc)))

Training Accuracy = 0.98


In [22]:
y_val_pred = model.predict_classes(X_val)
y_val_pred = enc.fit_transform(y_val_pred[:, np.newaxis])
correct_preds = np.sum(y_val == y_val_pred, axis=0)
val_acc = correct_preds / y_val.shape[0]
print("Val Accuracy = {:.2f}".format(np.mean(val_acc)))

Val Accuracy = 0.94


In [23]:
y_test_pred = model.predict_classes(X_test)
y_test_pred = enc.fit_transform(y_test_pred[:, np.newaxis])
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]
print("Test Accuracy = {:.2f}".format(np.mean(test_acc)))

Test Accuracy = 1.00


## 【問題5】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)

print(y.shape)
y = y[:, np.newaxis]
print(y.shape)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

(1460,)
(1460, 1)


In [26]:
K.clear_session()

#モデルを初期化
model = keras.models.Sequential()

#1つ目の隠れ層を追加
#12個の隠れユニットで構成
#入力層のためimput_dimの属性の値はデータセットの特徴量列の個数
model.add(keras.layers.Dense(5, input_dim=X_train.shape[1]))
#活性化関数はReLU
model.add(keras.layers.Activation('relu'))

#連続する層のunitsとinput_dimの個数を一致させる
model.add(keras.layers.Dense(y_train.shape[1], input_dim=5))
# model.add(keras.layers.Activation('softmax')) 

#オプティマイザとコスト関数を指定してモデルをコンパイル
model.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
#fitメソッドを呼び出してモデルのトレーニングを行う
model.fit(X_train, y_train, nb_epoch=20, batch_size=5, verbose=1)

C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
934/934 [==============================] - 0s 422us/step - loss: 39352886568.0171
Epoch 2/20
934/934 [==============================] - 0s 247us/step - loss: 39352623850.6210
Epoch 3/20
934/934 [==============================] - 0s 237us/step - loss: 39352233730.7409
Epoch 4/20
934/934 [==============================] - 0s 237us/step - loss: 39351700416.4111
Epoch 5/20
934/934 [==============================] - 0s 308us/step - loss: 39351020422.3041
Epoch 6/20
934/934 [==============================] - 0s 282us/step - loss: 39350190778.3812
Epoch 7/20
934/934 [==============================] - 0s 257us/step - loss: 39349215889.8158
Epoch 8/20
934/934 [==============================] - 0s 320us/step - loss: 39348093968.4454
Epoch 9/20
934/934 [==============================] - 0s 266us/step - loss: 39346819478.7495
Epoch 10/20
934/934 [==============================] - 0s 287us/step - loss: 39345355681.7131
Epoch 11/20
934/934 [==============================] - 0s 314us/step 

## 【問題6】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [28]:
from keras.datasets import mnist

In [29]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 平坦化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)

#正規化
X_train /= 255
X_test /= 255

# ワンホットライブラリのインスタンス作成
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.fit_transform(y_test[:, np.newaxis])

# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


In [30]:

K.clear_session()

#モデルを初期化
model = keras.models.Sequential()
#1つ目の隠れ層を追加
#12個の隠れユニットで構成
#入力層のためimput_dimの属性の値はデータセットの特徴量列の個数
model.add(keras.layers.Dense(15, input_dim=X_train.shape[1]))
#活性化関数はReLU
model.add(keras.layers.Activation('relu'))

#連続する層のunitsとinput_dimの個数を一致させる
model.add(keras.layers.Dense(y_train.shape[1], input_dim=12))
#活性化関数はSoftmax
model.add(keras.layers.Activation('softmax')) 

#オプティマイザとコスト関数を指定してモデルをコンパイル
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])

In [31]:
#fitメソッドを呼び出してモデルのトレーニングを行う
model.fit(X_train, y_train, nb_epoch=20, batch_size=5)

# モデル評価
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
48000/48000 [==============================] - 18s 375us/step - loss: 0.4502 - accuracy: 0.8717
Epoch 2/20
48000/48000 [==============================] - 19s 394us/step - loss: 0.2800 - accuracy: 0.9195
Epoch 3/20
48000/48000 [==============================] - 24s 496us/step - loss: 0.2397 - accuracy: 0.9306
Epoch 4/20
48000/48000 [==============================] - 24s 507us/step - loss: 0.2155 - accuracy: 0.9381
Epoch 5/20
48000/48000 [==============================] - 24s 501us/step - loss: 0.1984 - accuracy: 0.9426
Epoch 6/20
48000/48000 [==============================] - 19s 399us/step - loss: 0.1863 - accuracy: 0.9455
Epoch 7/20
48000/48000 [==============================] - 18s 377us/step - loss: 0.1767 - accuracy: 0.9489
Epoch 8/20
48000/48000 [==============================] - 16s 338us/step - loss: 0.1690 - accuracy: 0.9499
Epoch 9/20
48000/48000 [==============================] - 17s 346us/step - loss: 0.1627 - accuracy: 0.9520
Epoch 10/20
48000/48000 [============